Goal of analysis script is to take csv file of acoustic startle data (generated from the 'concatenate' function of SR Labs AS GUI) and analyze each AS block for each animal in the file. Five different blocks (with varrying numbers of trials). Result is excel file with a tab for each block.

In [1]:
import numpy as np

Use tkinter to open filedialog prompting user to select file to analyze.

In [2]:
from tkinter import filedialog
from tkinter import *
 
root = Tk()
root.filename =  filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("jpeg files","*.jpg"),("all files","*.*")))
print (root.filename)

C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Excel_files/AcousticStartle/18 7 12 timecourse pre.csv


In [3]:
fulldata = np.genfromtxt(root.filename, delimiter=",", dtype="U8")

print(fulldata[0])
print(len(fulldata))

['7/11/201' '14:28:41' ' AbbieFu' '338' ' No Entr' 'No Entry' '       1'
 '110dBAS ' '      1 ' '1' '0' '430' '16' '36']
6408


Generate list of animal numbers for iteration use **animal_numbers[i]

In [4]:
animal_numbers = []
for row in fulldata:
    if row[3] in animal_numbers:
        animal_numbers
    else: 
        animal_numbers.append(row[3])
        
print(animal_numbers)

['338', '339', '340', '320', '321', '322', '353', '354', '355', '323', '324', '325', '326', '327', '328', '329', '330', '331', '332', '333', '334', '335', '336', '337', '341', '342', '343', '344', '345', '346', '347', '348', '349', '350', '351', '352']


Parse data for animals into separate data sets (1 per animal) for iteration use **animal_data[animal_numbers[i]]

In [5]:
data = {}
i = 0
while i < len(animal_numbers):
    print(i)
    animal = []
    for row in fulldata:
        if row[3] == animal_numbers[i]:
            animal.append(row)
    data[animal_numbers[i]] = animal
    i = i + 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35


Find average basal startle level (assessed once per each two min break between blocks). Basal startle level = animal_numbers[0][50][7]. Search for basal startle level parameter by searching for row[7] (parameter name).

In [6]:
i = 0
twobreak_final = {}
while i < len(animal_numbers):
    print(i)
    sum = 0
    for row in data[animal_numbers[i]]:
        if row[7] == data[animal_numbers[0]][50][7]:
            sum = sum + np.int(row[11])
    ave = sum/4
    twobreak_final[animal_numbers[i]] = ave
    i = i + 1  

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35


Block 1 is 50 trials of the same parameter (110dB). To assess habituation across trials, first bin by 10 trials and then put each animal's data into an array. 

In [8]:
i = 0
block1ave_final = {}
while i < len(animal_numbers):
    print(i)
    j = 0
    ave_list = []
    k = 0
    sum = 0
    while j < 50:
        while k < 10:
            sum = sum + np.int((data[animal_numbers[i]][j][11]))
            k = k + 1
            j = j + 1
        ave = sum/10
        ave_list.append(ave)
        k = 0
        sum = 0
    block1ave_final[animal_numbers[i]] = ave_list
    i = i + 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35


Blocks 2, 3, and 4 contain multiple trials of multiple different parameters (pesudo-randomly assigned to trials). General strategy is to generate list of parameters for the block. Find average of each parameter for each animal. Block 2 assesses 9 different dB levels. Blocks 3 and 4 are PPI blocks, so after finding average of each parameter, need to also compute PPI.

Generate a list of block_2 parameters

In [9]:
j = 0
block2_param = []
for row in data[animal_numbers[1]][51:105]:
    if row[7] in block2_param:
        j = j + 1
    else: 
        block2_param.append(row[7])
        j = j + 1
    
print(block2_param)

['100dBAS ', '95dBAS  ', '90dBAS  ', '120dBAS ', '75dBAS  ', '85dBAS  ', '70dBAS  ', '80dBAS  ', '110dBAS ']


Block 2 is 54 trials of 9 different parameters, so final data point is a dictonary of animal numbers and within each animal number dictonary is a dictionary of block 2 parameters and the average output for each.


In [10]:
i = 0
block2_final = {}
while i < len(animal_numbers):
    print(i)
    j = 0
    param_sum = {}
    block2_ave = {}
    while j < len(block2_param):
        param_sum[block2_param[j]] = 0
        for row in data[animal_numbers[i]][51:105]:
            if row[7] == block2_param[j]:
                param_sum[block2_param[j]] = param_sum[block2_param[j]] + np.int(row[11])
        ave = param_sum[block2_param[j]]/6
        block2_ave[block2_param[j]] = ave
        j = j + 1
    block2_final[animal_numbers[i]] = block2_ave
    i = i + 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35


Generate a list of block_3 parameters

In [11]:
j = 0
block3_param = []
for row in data[animal_numbers[1]][106:135]:
    if row[7] in block3_param:
        j = j + 1
    else: 
        block3_param.append(row[7])
        j = j + 1
    
print(block3_param)

['80b4110A', '70b4110A', '75b4110A', '85b4120A', '75b4120A', '85b4110A']


Block 3 assesses PPI and is 40 trials of 6 different parameters. First find average of each parameter. 

In [12]:
i = 0
block3_semifinal = {}
while i < len(animal_numbers):
    print(i)
    j = 0
    param_sum = {}
    block3_ave = {}
    while j < len(block3_param):
        param_sum[block3_param[j]] = 0
        for row in data[animal_numbers[i]][106:135]:
            if row[7] == block3_param[j]:
                param_sum[block3_param[j]] = param_sum[block3_param[j]] + np.int(row[11])
        ave = param_sum[block3_param[j]]/5
        block3_ave[block3_param[j]] = ave
        j = j + 1
    block3_semifinal[animal_numbers[i]] = block3_ave
    i = i + 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35


Loop through animal numbers to compute PPI for each block 3 parameter (using block 2 pulse data)

In [13]:
i = 0
block3_final = {}
while i < len(animal_numbers):
    print(i)
    block3_ave = {}
    block3_ave[block3_param[0]] = np.float(block3_semifinal[animal_numbers[i]][block3_param[0]]) / np.float(block2_final[animal_numbers[i]][block2_param[8]])
    block3_ave[block3_param[1]] = np.float(block3_semifinal[animal_numbers[i]][block3_param[1]]) / np.float(block2_final[animal_numbers[i]][block2_param[8]])
    block3_ave[block3_param[2]] = np.float(block3_semifinal[animal_numbers[i]][block3_param[2]]) / np.float(block2_final[animal_numbers[i]][block2_param[8]])
    block3_ave[block3_param[5]] = np.float(block3_semifinal[animal_numbers[i]][block3_param[5]]) / np.float(block2_final[animal_numbers[i]][block2_param[8]])
    block3_ave[block3_param[3]] = np.float(block3_semifinal[animal_numbers[i]][block3_param[3]]) / np.float(block2_final[animal_numbers[i]][block2_param[3]])
    block3_ave[block3_param[4]] = np.float(block3_semifinal[animal_numbers[i]][block3_param[4]]) / np.float(block2_final[animal_numbers[i]][block2_param[3]])
    block3_final[animal_numbers[i]] = block3_ave
    i = i + 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35


Generate a list of block_4 parameters

In [15]:
j = 0
block4_param = []
for row in data[animal_numbers[1]][137:177]:
    if row[7] in block4_param:
        j = j + 1
    else: 
        block4_param.append(row[7])
        j = j + 1
    
print(block4_param)

['200msAS ', '20msAS  ', '100msAS ', '50msAS  ', '2secAS  ', '1000msAS', '500msAS ', '10msAS  ']


Block 4 assesses PPI and is 40 trials of 8 different parameters. First find average of each parameter. 

In [16]:
i = 0
block4_semifinal = {}
while i < len(animal_numbers):
    print(i)
    j = 0
    param_sum = {}
    block4_ave = {}
    while j < len(block4_param):
        param_sum[block4_param[j]] = 0
        for row in data[animal_numbers[i]][137:177]:
            if row[7] == block4_param[j]:
                param_sum[block4_param[j]] = param_sum[block4_param[j]] + np.int(row[11])
        ave = param_sum[block4_param[j]]/5
        block4_ave[block4_param[j]] = ave
        j = j + 1
    block4_semifinal[animal_numbers[i]] = block4_ave
    i = i + 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35


Loop through animal numbers to compute PPI for each block 4 parameter (using block 2 pulse data)

In [17]:
i = 0
block4_final = {}
while i < len(animal_numbers):
    print(i)
    j = 0
    block4_ave = {}
    while j < len(block4_param):
        block4_ave[block4_param[j]] = np.float(block4_semifinal[animal_numbers[i]][block4_param[j]]) / np.float(block2_final[animal_numbers[i]][block2_param[8]])
        j = j + 1
    block4_final[animal_numbers[i]] = block4_ave
    i = i + 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35


Combine blocks with only one data point per animal into an array and put array into dictionary stored via animal

In [18]:
i = 0
aveblocks_final = {}
aveblocks = [twobreak_final]
while i < len(animal_numbers):
    print(i)
    j = 0
    ave_final = []
    while j < len(aveblocks):
        ave_final.append(aveblocks[j][animal_numbers[i]])
        j = j + 1
    aveblocks_final[animal_numbers[i]] = ave_final
    i = i + 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35


Create final lists of data and block names.

In [19]:
final = [aveblocks_final, block1ave_final, block2_final, block3_final, block4_final]
finalnames = ['aveblocks_final', 'block1ave_final', 'block2_final', 'block3_final', 'block4_final']

Use pandas to convert each final block data into a dataframe. Use ExcelWriter to export each data frame to a separate tab in an excel workboook.

In [20]:
import pandas as pd
writer = pd.ExcelWriter(root.filename + '.analyzed.xlsx', engine='xlsxwriter')
i = 0
for row in final:
    print(i)
    df = pd.DataFrame(data=row)
    df.to_excel(writer, sheet_name=finalnames[i])
    i = i + 1
writer.save()

0
1
2
3
4
